In [589]:
import json
import jsonlines
from json_logic import jsonLogic
from json_logic.builtins import BUILTINS, op_var, op_log
from functools import reduce

In [590]:
def courses(data=None, *_ignored):
    return list(filter(None, op_var(data, "courses")))


def course(data=None, code="", number="", *_ignored):
    for course in courses(data):
        if course["code"] == code and course["number"] == number:
            return course
    return None


def units(data=None, *courses):
    flatten_list = lambda *lists: reduce(lambda a, b: a + b, lists, [])
    courses = flatten_list(*courses)
    
    total = 0
    for course in filter(None, courses):
        total += course["units"]
    return total

def consent(data=None, *consenter):
    op_log(data, f"Consent of: {', '.join(consenter) if consenter else 'None'}")
    return True

ops = {
    **BUILTINS,
    "courses": courses,
    "course": course,
    "units": units,
    "consent": consent,
}

# Examples


In [591]:
NUMBER_OF_EXAMPLES = 5

ors = [None] * NUMBER_OF_EXAMPLES
rules = [None] * NUMBER_OF_EXAMPLES
datas = [None] * NUMBER_OF_EXAMPLES

## Example 1

In [592]:
ors[0] = {
    "cid": "161622",
    "code": "CPSC",
    "number": "530",
    "faculty": "SC",
    "departments": ["CPSC"],
    "units": 3.0,
    "prereq": "CPSC 351; or 3 units from CPSC 219, 233 or 235 and 3 units from MATH 271, 273, 315 and 3 units from STAT 205, 213, 321.",
    "antireq": "Credit for CPSC 530 and 630 will not be allowed.",
    "coreq": None,
}

In [593]:
rules[0] = {
    "cid": "161622",
    "prereq": {
        "or": [
            {"course": ["CPSC", "351"]},
            {
                "and": [
                    {
                        ">=": [
                            {
                                "units": [
                                    {"course": ["CPSC", "219"]},
                                    {"course": ["CPSC", "233"]},
                                    {"course": ["CPSC", "235"]},
                                ]
                            },
                            3,
                        ]
                    },
                    {
                        ">=": [
                            {
                                "units": [
                                    {"course": ["MATH", "271"]},
                                    {"course": ["MATH", "273"]},
                                    {"course": ["MATH", "315"]},
                                ]
                            },
                            3,
                        ]
                    },
                    {
                        ">=": [
                            {
                                "units": [
                                    {"course": ["STAT", "205"]},
                                    {"course": ["STAT", "213"]},
                                    {"course": ["STAT", "321"]},
                                ]
                            },
                            3,
                        ]
                    },
                ]
            },
        ]
    },
    "antireq": {
        "or": [
            {"course": ["CPSC", "530"]},
            {"course": ["CPSC", "630"]},
        ]
    },
    "coreq": None,
}

In [594]:
datas[0] = {
    "courses": [
        {"code": "CPSC", "number": "329", "gpv": 2.7, "semester": "W2023", "units": 3.0, "faculty": "SC", "departments": ["CPSC"]},
        {"code": "CPSC", "number": "526", "gpv": 2.7, "semester": "F2023", "units": 6.0, "faculty": "SC", "departments": ["CPSC"]},
        {"code": "CPSC", "number": "219", "gpv": 2.7, "semester": "F2023", "units": 3.0, "faculty": "SC", "departments": ["CPSC"]},
        {"code": "MATH", "number": "313", "gpv": 2.7, "semester": "F2023", "units": 3.0, "faculty": "SC", "departments": ["CPSC"]},
        {"code": "MATH", "number": "271", "gpv": 2.7, "semester": "F2023", "units": 3.0, "faculty": "SC", "departments": ["CPSC"]},
        {"code": "STAT", "number": "205", "gpv": 2.7, "semester": "F2023", "units": 3.0, "faculty": "SC", "departments": ["CPSC"]},
        {"code": "CPSC", "number": "351", "gpv": 2.7, "semester": "F2023", "units": 3.0, "faculty": "SC", "departments": ["CPSC"]},
    ],
    "programs": ["ANMEBAH", "SPAN-MIN"],
}

a = jsonLogic(rules[0]["prereq"], datas[0], ops)
b = jsonLogic(rules[0]["antireq"], datas[0], ops)

bool(a), bool(b)

(True, False)

## Example 2

In [595]:
ors[1] = {
    "cid": "160417",
    "code": "CPSC",
    "number": "219",
    "faculty": "SC",
    "departments": ["CPSC"],
    "units": 3.0,
    "prereq": "CPSC 217 or DATA 211.",
    "antireq": "Credit for CPSC 219 and any of 233, 235, ENEL 497 or ENCM 493 will not be allowed.",
    "coreq": None,
}

In [596]:
rules[1] = {
    "cid": "160417",
    "prereq": {
        "or": [
            {"course": ["CPSC", "217"]}, 
            {"course": ["DATA", "211"]}
        ]
    },
    "antireq": {
        "and": [
            {"course": ["CPSC", "219"]},
            {
                "or": [
                    [
                        {"course": ["CPSC", "233"]},
                        {"course": ["CPSC", "235"]},
                        {"course": ["ENEL", "497"]},
                        {"course": ["ENCM", "493"]},
                    ],
                    True
                ]
            },
        ]
    },
    "coreq": None,
}

In [597]:
datas[1] = {
    "courses": [
        {"code": "CPSC", "number": "217", "gpv": 2.7, "semester": "F2023", "units": 3.0, "faculty": "SC", "departments": ["CPSC"]},
        {"code": "CPSC", "number": "219", "gpv": 2.7, "semester": "F2023", "units": 3.0, "faculty": "SC", "departments": ["CPSC"]},
        {"code": "CPSC", "number": "233", "gpv": 2.7, "semester": "F2023", "units": 6.0, "faculty": "SC", "departments": ["CPSC"]},
    ],
    "programs": ["ANMEBAH", "SPAN-MIN"],
}

a = jsonLogic(rules[1]["prereq"], datas[1], operations=ops)
b = jsonLogic(rules[1]["antireq"], datas[1], operations=ops)

bool(a), bool(b)

(True, True)

## Example 3

In [598]:
ors[2] = {
    "cid": "164005",
    "code": "CPSC",
    "number": "405",
    "faculty": "SC",
    "departments": ["CPSC"],
    "units": 3.0,
    "prereq": "SENG 300 or 301.",
    "antireq": "Credit for CPSC 405 and ENTI 415 will not be allowed.",
    "coreq": None,
}

In [599]:
rules[2] = {
    "cid": "164005",
    "prereq": {
        "or": [
            {"course": ["SENG", "300"]}, 
            {"course": ["SENG", "301"]}
        ]
    },
    "antireq": {
        "or": [
            {"course": ["CPSC", "405"]},
            {"course": ["ENTI", "415"]},
        ]
    },
    "coreq": None,
}

In [600]:
datas[2] = {
    "courses": [
        {"code": "SENG", "number": "300", "gpv": 2.7, "semester": "F2023", "units": 3.0, "faculty": "SC", "departments": ["CPSC"]},
        {"code": "CPSC", "number": "405", "gpv": 2.7, "semester": "F2023", "units": 3.0, "faculty": "SC", "departments": ["CPSC"]},
    ],
    "programs": ["ANMEBAH", "SPAN-MIN"],
}

a = jsonLogic(rules[2]["prereq"], datas[2], operations=ops)
b = jsonLogic(rules[2]["antireq"], datas[2], operations=ops)


bool(a), bool(b)

(True, True)

## Example 4

In [601]:
ors[3] = {
    "cid": "107275",
    "code": "CPSC",
    "number": "550B",
    "faculty": "SC",
    "departments": ["CPSC"],
    "units": 6.0,
    "prereq": "CPSC 441, 457 and consent of the Department.",
    "antireq": None,
    "coreq": None,
}

In [602]:
rules[3] = {
    "cid": "107275",
    "prereq": {
        "and": [
            {"course": ["CPSC", "441"]},
            {"course": ["CPSC", "457"]},
            {"consent": ["CPSC"]},
        ]
    },
    "antireq": None,
    "coreq": None,
}

In [603]:
datas[3] = {
    "courses": [
        {"code": "CPSC", "number": "441", "gpv": 2.7, "semester": "F2023", "units": 3.0, "faculty": "SC", "departments": ["CPSC"]},
        {"code": "CPSC", "number": "457", "gpv": 2.7, "semester": "F2023", "units": 3.0, "faculty": "SC", "departments": ["CPSC"]},
    ],
    "programs": ["ANMEBAH", "SPAN-MIN"],
}

a = jsonLogic(rules[3]["prereq"], datas[3], operations=ops)
b = jsonLogic(rules[3]["antireq"], datas[3], operations=ops)


bool(a), bool(b)

"Consent of: CPSC"


(True, False)

## Example 5

In [604]:
ors[4] = {
    "cid": "161195",
    "code": "CPSC",
    "number": "449",
    "faculty": "SC",
    "departments": ["CPSC"],
    "units": 3.0,
    "prereq": "CPSC 319 or 331; CPSC 313 or 351; and PHIL 279 or 377.",
    "antireq": "Credit for CPSC 449 and 349 will not be allowed.",
    "coreq": None,
}

In [605]:
rules[4] = {
    "cid": "161195",
    "prereq": {
        "and": [
            {
                "or": [
                    {"course": ["CPSC", "319"]},
                    {"course": ["CPSC", "331"]},
                ]
            },
            {
                "or": [
                    {"course": ["CPSC", "313"]},
                    {"course": ["CPSC", "351"]},
                ]
            },
            {
                "or": [
                    {"course": ["PHIL", "279"]},
                    {"course": ["PHIL", "377"]},
                ]
            },
        ]
    },
    "antireq": {
        "or": [
            {"course": ["CPSC", "449"]},
            {"course": ["CPSC", "349"]},
        ]
    },
    "coreq": None,
}

In [606]:
datas[4] = {
    "courses": [
        {"code": "CPSC", "number": "319", "gpv": 2.7, "semester": "F2023", "units": 3.0, "faculty": "SC", "departments": ["CPSC"]},
        {"code": "CPSC", "number": "351", "gpv": 2.7, "semester": "F2023", "units": 3.0, "faculty": "SC", "departments": ["CPSC"]},
        {"code": "PHIL", "number": "279", "gpv": 2.7, "semester": "F2023", "units": 3.0, "faculty": "SC", "departments": ["CPSC"]},
    ],
    "programs": ["ANMEBAH", "SPAN-MIN"],
}

a = jsonLogic(rules[4]["prereq"], datas[4], operations=ops)
b = jsonLogic(rules[4]["antireq"], datas[4], operations=ops)


bool(a), bool(b)

(True, False)

# Aggregation

In [607]:
SYSTEM_MESSAGE = {
    "role": "system",
    "content": "Nero is a data conversion engine, from textual to structural.",
}

In [608]:
# Write to jsonlines file
with jsonlines.open("train/train.jsonl", "w") as writer:
    for t in range(NUMBER_OF_EXAMPLES):
        messages = [SYSTEM_MESSAGE]

        messages.append({
            "role": "user",
            "content": json.dumps(ors[t]),
        })

        messages.append({
            "role": "assistant",
            "content": json.dumps(rules[t]),
        })

        l = {
            "messages": messages,
        }

        writer.write(l)
    